In [1]:
# 파일이름 : model_evaluation_same_models.ipynb
# 코드설명 : 물성값 및 불량여부 예측 AI 모델의 성능 비교 및 최적(동일 기술) 모델 선정, 필요없는 모델 제거
# 입/출력 : 예측 대상별 학습된 모델들 / 예측 대상별 최종 선정 모델(동일 기술)
# 유의 사항 :
# 1. 경량화에서 실수로 제거 시 재학습에 오랜 시간이 소요되므로, 복사해두고 진행
# 2. autogluon(0.8.2)과 python(3.9.18) 버전을 맞추어야 함
# 3. 모델 저장 경로를 잘 수정하여 진행
# 최종수정 : 2023년 11월 27일
# 제 작 자 : 홍민성 (mshong@micube.co.kr), 맹영준 (myj6223@micube.co.kr)
# Copyright : MICUBE Solution, Inc.

# python 버전 : 3.9.18
# autogluon 버전 : 0.8.2

In [2]:
# 관련 라이브러리 로드
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", 500)
pd.set_option('display.max_rows', 50)

import os
import random
import warnings
warnings.filterwarnings(action='ignore')

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error, recall_score, accuracy_score

# autoML
from autogluon.tabular import TabularPredictor

# 동일 결과 재생성을 위한 랜덤성 고정
seed = 42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)

import time

In [3]:
# SMAPE 계산 함수 정의
def smape_cal(y_true, y_pred):
    return np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

# 0을 제외한 MAPE 계산 함수 정의
def mape_non_zero(y_true, y_pred):
    mask = y_true != 0
    return np.mean(np.abs((y_pred[mask] - y_true[mask]) / y_true[mask]))

## 1. AutoML (Autogluon) 회귀 모델 로드 및 모델 조정

### 1) 물성값 예측에 대한 회귀 모델 로드 및 leaderboard 출력

In [3]:
# 물성값: regression, 불량여부: binary
problem_type_ = 'regression' # 'regression', 'binary'

# 모델 인덱스
file_suffix='combined'

if problem_type_ == 'regression':
    # 물성값 예측
    # yCols=['HS','SG','TS','EB']#'MNY','REHO_MIN','REHO_MAX','REHO_TS2','REHO_TC90','SCR']
    # yCols=['MNY','REHO_MIN','REHO_MAX','REHO_TS2','REHO_TC90','SCR']    
    yCols=['HS','SG','TS','EB','MNY_MIN','MNY_T5','MNY_T35','REHO_MIN','REHO_MAX','REHO_TS2','REHO_TC90']
    # yCols=['REHO_MIN']#,'REHO_TS2']
    data_type_ = 'SFT_regr'
else:
    # 불량여부 예측
    # yCols=['HS_RESULT','SG_RESULT','TS_RESULT','EB_RESULT']#,'MNY_RESULT','REHO_RESULT','SCR_RESULT']
    yCols=['MNY_RESULT','REHO_RESULT','SCR_RESULT']
    data_type_ = 'SFT_clas'

for yCol in yCols:
    print(f'{yCol}!!!!')
    print(f'='*80)
    # 데이터셋 로드 및 모델 입출력 데이터 생성
    train = pd.read_csv(f'./final_dataset/{file_suffix}_data_{yCol}_fin.csv')
    X_train, X_test, y_train, y_test = train_test_split(train.drop(['ID', 'REAL_VAL','TYPE'], axis=1), train['REAL_VAL'], test_size=0.2, random_state=seed)
    train_data = train.iloc[X_train.index].drop(['ID','TYPE'], axis=1)

    # 모델 로드    
    predictor = TabularPredictor.load(path=f'{data_type_}/{file_suffix}_{yCol}_models', verbosity = 0)

    # 테스트 세트에 대한 예측
    y_pred = predictor.predict(X_test)

    res_df = pd.DataFrame()
    res_df[yCol+'_real'] = y_test
    res_df[yCol+'_pred'] = y_pred

    # 성능 평가
    if problem_type_ == 'regression':
        mape = mean_absolute_percentage_error(res_df[yCol+'_real'], res_df[yCol+'_pred'])
        zero_mape = mape_non_zero(res_df[yCol+'_real'], res_df[yCol+'_pred'])
        smape = smape_cal(res_df[yCol+'_real'], res_df[yCol+'_pred'])
        print(f"{yCol} >> MAPE: {mape:.4f}, zero_mape: {zero_mape:.4f}, smape: {smape:.4f}")
    else:
        recall = recall_score(res_df[yCol+'_real'], res_df[yCol+'_pred'])
        accuracy = accuracy_score(res_df[yCol+'_real'], res_df[yCol+'_pred'])
        print(f"{yCol} >> reall: {recall:.4f}, accuracy: {accuracy:.4f}")

    # 학습된 모델의 성능 요약
    display(predictor.leaderboard())
    print(f'='*80,'\n')

HS!!!!
HS >> MAPE: 0.0204, zero_mape: 0.0204, smape: 0.0202


,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-2.279489,root_mean_squared_error,20.926387,158.957199,0.001005,0.030382,3,True,10
1,RandomForestMSE_BAG_L2,-2.313382,root_mean_squared_error,20.433304,127.639054,0.952536,21.959915,2,True,8
2,LightGBM_BAG_L2,-2.341334,root_mean_squared_error,19.623910,109.549438,0.143143,3.870299,2,True,7
3,CatBoost_BAG_L2,-2.366563,root_mean_squared_error,19.646750,129.094756,0.165983,23.415616,2,True,9
4,LightGBMXT_BAG_L2,-2.386529,root_mean_squared_error,19.663720,109.680987,0.182952,4.001848,2,True,6
5,WeightedEnsemble_L2,-2.404370,root_mean_squared_error,16.801508,105.502207,0.000000,0.022247,2,True,5
6,LightGBMXT_BAG_L1,-2.471288,root_mean_squared_error,13.198840,90.275082,13.198840,90.275082,1,True,3
7,LightGBM_BAG_L1,-2.485216,root_mean_squared_error,1.040023,15.060994,1.040023,15.060994,1,True,4
8,KNeighborsDist_BAG_L1,-4.032636,root_mean_squared_error,2.562646,0.143883,2.562646,0.143883,1,True,2
9,KNeighborsUnif_BAG_L1,-4.429603,root_mean_squared_error,2.679259,0.199180,2.679259,0.199180,1,True,1



SG!!!!
SG >> MAPE: 0.0060, zero_mape: 0.0060, smape: 0.0060


,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-0.014515,root_mean_squared_error,24.425097,145.867272,0.002000,0.055708,3,True,10
1,LightGBM_BAG_L2,-0.014564,root_mean_squared_error,23.630896,105.969613,0.220840,4.399409,2,True,8
2,CatBoost_BAG_L2,-0.014781,root_mean_squared_error,23.657391,132.330566,0.247335,30.760362,2,True,9
3,LightGBMXT_BAG_L2,-0.014801,root_mean_squared_error,23.954922,110.651793,0.544866,9.081589,2,True,7
4,WeightedEnsemble_L2,-0.014943,root_mean_squared_error,14.998777,94.890521,0.001000,0.042429,2,True,6
5,LightGBMXT_BAG_L1,-0.015132,root_mean_squared_error,3.980555,48.982494,3.980555,48.982494,1,True,3
6,LightGBM_BAG_L1,-0.015179,root_mean_squared_error,3.065727,45.574574,3.065727,45.574574,1,True,4
7,KNeighborsDist_BAG_L1,-0.032003,root_mean_squared_error,7.951496,0.291023,7.951496,0.291023,1,True,2
8,KNeighborsUnif_BAG_L1,-0.034000,root_mean_squared_error,8.161548,0.276830,8.161548,0.276830,1,True,1
9,CatBoost_BAG_L1,-0.053197,root_mean_squared_error,0.250731,6.445283,0.250731,6.445283,1,True,5



TS!!!!
TS >> MAPE: 0.0546, zero_mape: 0.0546, smape: 0.0530


...

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT_BAG_L2,-2.914455,root_mean_squared_error,106.049371,82.725281,2.910006,55.583918,2,True,4
1,WeightedEnsemble_L3,-2.914455,root_mean_squared_error,106.051876,82.948537,0.002505,0.223256,3,True,5
2,WeightedEnsemble_L2,-3.097516,root_mean_squared_error,103.143365,27.281588,0.004000,0.140225,2,True,3
3,KNeighborsUnif_BAG_L1,-3.262869,root_mean_squared_error,102.118145,1.172523,102.118145,1.172523,1,True,1
4,LightGBMXT_BAG_L1,-3.395194,root_mean_squared_error,1.021219,25.968841,1.021219,25.968841,1,True,2



REHO_TS2!!!!
REHO_TS2 >> MAPE: 0.0525, zero_mape: 0.0525, smape: 0.0517


,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-4.815563,root_mean_squared_error,83.330220,107.596982,0.003013,0.180204,3,True,5
1,LightGBMXT_BAG_L2,-4.816755,root_mean_squared_error,83.327206,107.416777,3.132507,59.030411,2,True,4
2,WeightedEnsemble_L2,-4.998716,root_mean_squared_error,80.197714,48.523976,0.003015,0.137610,2,True,3
3,LightGBMXT_BAG_L1,-5.115299,root_mean_squared_error,2.389072,47.561206,2.389072,47.561206,1,True,2
4,KNeighborsUnif_BAG_L1,-5.672625,root_mean_squared_error,77.805627,0.825161,77.805627,0.825161,1,True,1



REHO_TC90!!!!
REHO_TC90 >> MAPE: 0.0518, zero_mape: 0.0518, smape: 0.0512


,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-11.124349,root_mean_squared_error,95.448385,97.048529,0.003066,0.171310,3,True,5
1,LightGBMXT_BAG_L2,-11.125567,root_mean_squared_error,95.445319,96.877219,2.629518,55.477272,2,True,4
2,WeightedEnsemble_L2,-11.576809,root_mean_squared_error,92.819812,41.554165,0.004011,0.154217,2,True,3
3,LightGBMXT_BAG_L1,-11.901785,root_mean_squared_error,1.687362,40.599977,1.687362,40.599977,1,True,2
4,KNeighborsUnif_BAG_L1,-13.119707,root_mean_squared_error,91.128439,0.799970,91.128439,0.799970,1,True,1


### 2) 모델 선정

In [4]:
# 선정된 모델명 리스트
trg_model_ls = ['WeightedEnsemble_L3']
# trg_model_ls = ['LightGBMXT_BAG_L1','KNeighborsUnif_BAG_L1']
print(f'선정된 모델 : {trg_model_ls}')

선정된 모델 : ['WeightedEnsemble_L3']


### 3) 개별(LAB, MES-CMB, MES-FMB) 데이터의 모델 선정 및 성능 평가 요약

In [6]:
%%time
# 물성값: regression, 불량여부: binary
problem_type_ = 'regression' # 'regression', 'binary'

# 모델 인덱스
file_suffix='combined'

if problem_type_ == 'regression':
    # 물성값 예측
    # yCols=['HS','SG','TS','EB']#'MNY','REHO_MIN','REHO_MAX','REHO_TS2','REHO_TC90','SCR']
    #yCols=['HS','SG','TS','EB','REHO_MIN','REHO_MAX','REHO_TS2','REHO_TC90','MNY_MIN','MNY_T5','MNY_T35']
    yCols=['HS','SG','TS','EB','MNY_MIN','MNY_T5','MNY_T35','REHO_MIN','REHO_MAX','REHO_TS2','REHO_TC90']

    data_type_ = 'SFT_regr'
    
    col_Nms = ['type','model_nm','target_y']
    col_Nms = col_Nms + ['mape','mape0','smape']
else:
    # 불량여부 예측
    # yCols=['HS_RESULT','SG_RESULT','TS_RESULT','EB_RESULT']#,'MNY_RESULT','REHO_RESULT','SCR_RESULT']
    yCols=['MNY_RESULT','REHO_RESULT','SCR_RESULT']
    data_type_ = 'SFT_clas'
    
    col_Nms = ['type','model_nm','target_y']
    col_Nms = col_Nms + ['recall','accuracy']
    
col_Nms.append('데이터수')
col_Nms.append('예측시간')
fin_df = pd.DataFrame(columns=col_Nms)

for yCol in yCols:    
    print(f'{yCol}!!!!')
    print(f'='*80)
    # 데이터셋 로드 및 모델 입출력 데이터 생성
    train = pd.read_csv(f'./final_dataset/{file_suffix}_data_{yCol}_fin.csv')
    X_train, X_test, y_train, y_test = train_test_split(train.drop(['ID', 'REAL_VAL'], axis=1), train[['REAL_VAL','TYPE']], test_size=0.2, random_state=seed)

    # 모델 로드    
    predictor = TabularPredictor.load(path=f'{data_type_}/{file_suffix}_{yCol}_models', verbosity = 0)
        
    types = X_test['TYPE'].unique().tolist()
    types.sort()
    display(types)
    
    for type_ in types:
        tmp_X_test = X_test[X_test['TYPE'] == type_]
        tmp_y_test = y_test[y_test['TYPE'] == type_]
        tmp_X_test = tmp_X_test.drop(columns=['TYPE'])
        tmp_y_test = tmp_y_test['REAL_VAL']
        
        for model_name in trg_model_ls:
            result_ls = []
            # 테스트 세트에 대한 예측
            start = time.time()
            y_pred = predictor.predict(tmp_X_test, model=model_name)
            pred_time = np.round(time.time()-start, 2)

            res_df = pd.DataFrame()
            res_df[yCol+'_real'] = tmp_y_test
            res_df[yCol+'_pred'] = y_pred

            # 성능 평가
            if problem_type_ == 'regression':
                mape = mean_absolute_percentage_error(res_df[yCol+'_real'], res_df[yCol+'_pred'])
                zero_mape = mape_non_zero(res_df[yCol+'_real'], res_df[yCol+'_pred'])
                smape = smape_cal(res_df[yCol+'_real'], res_df[yCol+'_pred'])
                print(f"{model_name},{type_}: {yCol} >> MAPE: {mape:.4f}, zero_mape: {zero_mape:.4f}, smape: {smape:.4f}")
                result_ls = [mape, zero_mape, smape]
            else:
                recall = recall_score(res_df[yCol+'_real'], res_df[yCol+'_pred'])
                accuracy = accuracy_score(res_df[yCol+'_real'], res_df[yCol+'_pred'])
                print(f"{model_name},{type_}: {yCol} >> reall: {recall:.4f}, accuracy: {accuracy:.4f}")
                result_ls = [recall, accuracy, '']

            fin_df.loc[len(fin_df)] = [type_, model_name, yCol] + result_ls + [len(tmp_X_test)] + [pred_time]
            # predictor.leaderboard()

# 데이터 타입별 성능 결과 정렬
fin_df.sort_values(['model_nm','type']).to_csv('./evaluation_results/evluation_of_individuals.csv', index=False)
fin_df.sort_values(['model_nm','type'])

HS!!!!


['LAB', 'mCMB', 'mFMB']

WeightedEnsemble_L3,LAB: HS >> MAPE: 0.0603, zero_mape: 0.0603, smape: 0.0590
WeightedEnsemble_L3,mCMB: HS >> MAPE: 0.0190, zero_mape: 0.0190, smape: 0.0190
WeightedEnsemble_L3,mFMB: HS >> MAPE: 0.0154, zero_mape: 0.0154, smape: 0.0153
SG!!!!


['LAB', 'mCMB', 'mFMB']

WeightedEnsemble_L3,LAB: SG >> MAPE: 0.0186, zero_mape: 0.0186, smape: 0.0186
WeightedEnsemble_L3,mCMB: SG >> MAPE: 0.0059, zero_mape: 0.0059, smape: 0.0059
WeightedEnsemble_L3,mFMB: SG >> MAPE: 0.0044, zero_mape: 0.0044, smape: 0.0044
TS!!!!


...

['LAB', 'mCMB', 'mFMB']

WeightedEnsemble_L3,LAB: REHO_TS2 >> MAPE: 0.1647, zero_mape: 0.1647, smape: 0.1523
WeightedEnsemble_L3,mCMB: REHO_TS2 >> MAPE: 0.0694, zero_mape: 0.0694, smape: 0.0684
WeightedEnsemble_L3,mFMB: REHO_TS2 >> MAPE: 0.0496, zero_mape: 0.0496, smape: 0.0490
REHO_TC90!!!!


['LAB', 'mCMB', 'mFMB']

WeightedEnsemble_L3,LAB: REHO_TC90 >> MAPE: 0.1263, zero_mape: 0.1263, smape: 0.1199
WeightedEnsemble_L3,mCMB: REHO_TC90 >> MAPE: 0.0697, zero_mape: 0.0697, smape: 0.0679
WeightedEnsemble_L3,mFMB: REHO_TC90 >> MAPE: 0.0497, zero_mape: 0.0497, smape: 0.0493
CPU times: total: 20min 26s
Wall time: 3min 14s


,type,model_nm,target_y,mape,mape0,smape,데이터수,예측시간
0,LAB,WeightedEnsemble_L3,HS,0.060262,0.060262,0.058971,880,2.55
3,LAB,WeightedEnsemble_L3,SG,0.018628,0.018628,0.018597,820,1.27
6,LAB,WeightedEnsemble_L3,TS,0.113545,0.113545,0.104445,836,1.47
9,LAB,WeightedEnsemble_L3,EB,0.190129,0.190129,0.153579,869,1.83
12,LAB,WeightedEnsemble_L3,MNY_MIN,0.228303,0.228303,0.198804,669,1.09
15,LAB,WeightedEnsemble_L3,MNY_T5,0.238367,0.238367,0.207442,780,1.22
18,LAB,WeightedEnsemble_L3,MNY_T35,0.274751,0.274751,0.240876,670,1.45
21,LAB,WeightedEnsemble_L3,REHO_MIN,0.357385,0.357385,0.280579,756,1.01
24,LAB,WeightedEnsemble_L3,REHO_MAX,0.254412,0.254412,0.197077,891,0.95
27,LAB,WeightedEnsemble_L3,REHO_TS2,0.164650,0.164650,0.152296,872,0.90


### 4) 통합 데이터의 최적 예측 모델(들) 선정 및 모델별 성능 요약

In [7]:
# 물성값: regression, 불량여부: binary
problem_type_ = 'regression' # 'regression', 'binary'

# 모델 인덱스
file_suffix='combined'

if problem_type_ == 'regression':
    # 물성값 예측
    #yCols=['HS','SG','TS','EB']#'MNY','REHO_MIN','REHO_MAX','REHO_TS2','REHO_TC90','SCR']
    yCols=['HS','SG','TS','EB','MNY_MIN','MNY_T5','MNY_T35','REHO_MIN','REHO_MAX','REHO_TS2','REHO_TC90']

    # yCols=['MNY','REHO_MIN','REHO_MAX','REHO_TS2','REHO_TC90','SCR']
    data_type_ = 'SFT_regr'
    
    col_Nms = ['model_nm','target_y','mape','mape0','smape']
else:
    # 불량여부 예측
    yCols=['HS_RESULT','SG_RESULT','TS_RESULT','EB_RESULT']#,'MNY_RESULT','REHO_RESULT','SCR_RESULT']
    # yCols=['MNY_RESULT','REHO_RESULT','SCR_RESULT']
    data_type_ = 'SFT_clas'
    
    col_Nms = ['model_nm','target_y','recall','accuracy','']
    
col_Nms.append('데이터수')
col_Nms.append('예측시간')
fin_df = pd.DataFrame(columns=col_Nms)

for yCol in yCols:    
    print(f'{yCol}!!!!')
    print(f'='*80)
    # 데이터셋 로드 및 모델 입출력 데이터 생성
    train = pd.read_csv(f'./final_dataset/{file_suffix}_data_{yCol}_fin.csv')
    X_train, X_test, y_train, y_test = train_test_split(train.drop(['ID', 'REAL_VAL','TYPE'], axis=1), train[['REAL_VAL']], test_size=0.2, random_state=seed)

    # 모델 로드    
    predictor = TabularPredictor.load(path=f'{data_type_}/{file_suffix}_{yCol}_models', verbosity = 0)
    
    for model_name in trg_model_ls:
        # 테스트 세트에 대한 예측
        result_ls = []
        start = time.time()
        y_pred = predictor.predict(X_test, model=model_name)
        pred_time = np.round(time.time()-start, 2)

        res_df = pd.DataFrame()
        res_df[yCol+'_real'] = y_test
        res_df[yCol+'_pred'] = y_pred

        # 성능 평가
        if problem_type_ == 'regression':
            mape = mean_absolute_percentage_error(res_df[yCol+'_real'], res_df[yCol+'_pred'])
            zero_mape = mape_non_zero(res_df[yCol+'_real'], res_df[yCol+'_pred'])
            smape = smape_cal(res_df[yCol+'_real'], res_df[yCol+'_pred'])
            print(f"{model_name}, {yCol} >> MAPE: {mape:.4f}, zero_mape: {zero_mape:.4f}, smape: {smape:.4f}")
            result_ls = [mape, zero_mape, smape]
        else:
            recall = recall_score(res_df[yCol+'_real'], res_df[yCol+'_pred'])
            accuracy = accuracy_score(res_df[yCol+'_real'], res_df[yCol+'_pred'])
            print(f"{model_name}, {yCol} >> reall: {recall:.4f}, accuracy: {accuracy:.4f}")
            result_ls = [recall, accuracy, '']

        fin_df.loc[len(fin_df)] = [model_name, yCol] + result_ls + [len(X_test)] + [pred_time]
        # predictor.leaderboard()

# 데이터 타입별 성능 결과 정렬
fin_df.sort_values(['model_nm']).to_csv('./evaluation_results/evluation_of_combined.csv', index=False)
fin_df.sort_values(['model_nm'])

HS!!!!
WeightedEnsemble_L3, HS >> MAPE: 0.0204, zero_mape: 0.0204, smape: 0.0202
SG!!!!
WeightedEnsemble_L3, SG >> MAPE: 0.0060, zero_mape: 0.0060, smape: 0.0060
TS!!!!
WeightedEnsemble_L3, TS >> MAPE: 0.0546, zero_mape: 0.0546, smape: 0.0530
EB!!!!
WeightedEnsemble_L3, EB >> MAPE: 0.0729, zero_mape: 0.0729, smape: 0.0671
MNY_MIN!!!!
WeightedEnsemble_L3, MNY_MIN >> MAPE: 0.0787, zero_mape: 0.0787, smape: 0.0706
MNY_T5!!!!
WeightedEnsemble_L3, MNY_T5 >> MAPE: 0.0724, zero_mape: 0.0724, smape: 0.0695
MNY_T35!!!!
WeightedEnsemble_L3, MNY_T35 >> MAPE: 0.0782, zero_mape: 0.0782, smape: 0.0748
REHO_MIN!!!!
WeightedEnsemble_L3, REHO_MIN >> MAPE: 0.1571, zero_mape: 0.1571, smape: 0.1417
REHO_MAX!!!!
WeightedEnsemble_L3, REHO_MAX >> MAPE: 0.0815, zero_mape: 0.0815, smape: 0.0759
REHO_TS2!!!!
WeightedEnsemble_L3, REHO_TS2 >> MAPE: 0.0525, zero_mape: 0.0525, smape: 0.0517
REHO_TC90!!!!
WeightedEnsemble_L3, REHO_TC90 >> MAPE: 0.0518, zero_mape: 0.0518, smape: 0.0512


,model_nm,target_y,mape,mape0,smape,데이터수,예측시간
0,WeightedEnsemble_L3,HS,0.020398,0.020398,0.020219,9776,5.28
1,WeightedEnsemble_L3,SG,0.006002,0.006002,0.005986,17764,8.34
2,WeightedEnsemble_L3,TS,0.054562,0.054562,0.053034,9307,3.88
3,WeightedEnsemble_L3,EB,0.072939,0.072939,0.067080,9430,4.37
4,WeightedEnsemble_L3,MNY_MIN,0.078740,0.078740,0.070642,27666,14.52
5,WeightedEnsemble_L3,MNY_T5,0.072377,0.072377,0.069481,14563,6.35
6,WeightedEnsemble_L3,MNY_T35,0.078169,0.078169,0.074783,11531,4.48
7,WeightedEnsemble_L3,REHO_MIN,0.157071,0.157071,0.141671,61277,25.80
8,WeightedEnsemble_L3,REHO_MAX,0.081479,0.081479,0.075868,61628,26.12
9,WeightedEnsemble_L3,REHO_TS2,0.052482,0.052482,0.051675,54785,22.01


In [8]:
predictor.info()

{'path': 'SFT_regr/combined_REHO_TC90_models',
 'label': 'REAL_VAL',
 'random_state': 0,
 'version': '1.1.0',
 'features': ['INSP_TIME',
  'INSP_TEMP',
  'RPM1',
  'RPM2',
  'RPM3',
  'RPM4',
  'RPM5',
  'RPM6',
  'RPM7',
  'RPM8',
  'RPM9',
  'RPM10',
  'RPM11',
  'RPM12',
  'RPM13',
  'RPM14',
  'RPM15',
  'RPM16',
  'RPM17',
  'RPM18',
  'RPM19',
  'RPM20',
  'RPM21',
  'RPM22',
  'RPM23',
  'TEMP1',
  'TEMP2',
  'TEMP3',
  'TEMP4',
  'TEMP5',
  'TEMP6',
  'TEMP7',
  'TEMP8',
  'TEMP9',
  'TEMP10',
  'TEMP11',
  'TEMP12',
  'TEMP13',
  'TEMP14',
  'TEMP15',
  'TEMP16',
  'TEMP17',
  'TEMP18',
  'TEMP19',
  'TEMP20',
  'TEMP21',
  'TEMP22',
  'TEMP23',
  'TIME1',
  'TIME2',
  'TIME3',
  'TIME4',
  'TIME5',
  'TIME6',
  'TIME7',
  'TIME8',
  'TIME9',
  'TIME10',
  'TIME11',
  'TIME12',
  'TIME13',
  'TIME14',
  'TIME15',
  'TIME16',
  'TIME17',
  'TIME18',
  'TIME19',
  'TIME20',
  'TIME21',
  'TIME22',
  'TIME23',
  'JUK1',
  'JUK2',
  'JUK3',
  'JUK4',
  'JUK5',
  'JUK6',
  'JUK7',


### 5) 최종적으로 선정된 모델 외 삭제 (모델 로딩 시간 단축)
<p style="font-weight:bold"> <span style="color:red">** 주의 : 실수로 모델 제거시, 다시 학습해야하므로 복사 해두고 진행하기 바랍니다.</span> </p>

In [9]:
# 선정된 모델명 리스트
trg_model_ls = ['WeightedEnsemble_L3']
# trg_model_ls = ['LightGBMXT_BAG_L1','KNeighborsUnif_BAG_L1']
print(f'선정된 모델 : {trg_model_ls}')

선정된 모델 : ['WeightedEnsemble_L3']


In [10]:
# 물성값: regression, 불량여부: binary
problem_type_ = 'regression' # 'regression', 'binary'

# 모델 인덱스
file_suffix='combined'

if problem_type_ == 'regression':
    # 물성값 예측
    # yCols=['HS','SG','TS','EB']#'MNY','REHO_MIN','REHO_MAX','REHO_TS2','REHO_TC90','SCR']
    yCols=['HS','SG','TS','EB','REHO_MIN','REHO_MAX','REHO_TS2','REHO_TC90','MNY_MIN','MNY_T5','MNY_T35']
    data_type_ = 'SFT_regr'
else:
    # 불량여부 예측
    yCols=['HS_RESULT','SG_RESULT','TS_RESULT','EB_RESULT']#,'MNY_RESULT','REHO_RESULT','SCR_RESULT']
    yCols=['MNY_RESULT','REHO_RESULT','SCR_RESULT']
    data_type_ = 'SFT_clas'

for yCol in yCols:
    print(yCol)
    predictor = TabularPredictor.load(path=f'{data_type_}/{file_suffix}_{yCol}_models', verbosity = 0)
    predictor.delete_models(models_to_keep=trg_model_ls, models_to_delete=None, dry_run=False)
    predictor.leaderboard()

HS
SG
TS
EB
REHO_MIN
REHO_MAX
REHO_TS2
REHO_TC90
MNY_MIN
MNY_T5
MNY_T35


## 2. AutoML (Autogluon) 분류 모델 로드 및 모델 조정

### 1) 불량여부 예측에 대한 분류 모델 로드 및 leaderboard 출력

In [ ]:
# 물성값: regression, 불량여부: binary
problem_type_ = 'binary' # 'regression', 'binary'

# 모델 인덱스
file_suffix='combined'

if problem_type_ == 'regression':
    # 물성값 예측
    yCols=['HS','SG','TS','EB']#'MNY','REHO_MIN','REHO_MAX','REHO_TS2','REHO_TC90','SCR']
    yCols=['MNY','REHO_MIN','REHO_MAX','REHO_TS2','REHO_TC90','SCR']
    data_type_ = 'SFT_regr'
else:
    # 불량여부 예측
    yCols=['HS_RESULT','SG_RESULT','TS_RESULT','EB_RESULT']#,'MNY_RESULT','REHO_RESULT','SCR_RESULT']
    yCols=['MNY_RESULT','REHO_RESULT','SCR_RESULT']
    data_type_ = 'SFT_clas'

for yCol in yCols:
    print(f'{yCol}!!!!')
    print(f'='*80)
    # 데이터셋 로드 및 모델 입출력 데이터 생성
    train = pd.read_csv(f'./final_dataset/{file_suffix}_data_{yCol}_fin.csv')
    X_train, X_test, y_train, y_test = train_test_split(train.drop(['ID', 'REAL_VAL','TYPE'], axis=1), train['REAL_VAL'], test_size=0.2, random_state=seed)
    train_data = train.iloc[X_train.index].drop(['ID','TYPE'], axis=1)

    # 모델 로드    
    predictor = TabularPredictor.load(path=f'{data_type_}/{file_suffix}_{yCol}_models', verbosity = 0)

    # 테스트 세트에 대한 예측
    y_pred = predictor.predict(X_test)

    res_df = pd.DataFrame()
    res_df[yCol+'_real'] = y_test
    res_df[yCol+'_pred'] = y_pred

    # 성능 평가
    if problem_type_ == 'regression':
        mape = mean_absolute_percentage_error(res_df[yCol+'_real'], res_df[yCol+'_pred'])
        zero_mape = mape_non_zero(res_df[yCol+'_real'], res_df[yCol+'_pred'])
        smape = smape_cal(res_df[yCol+'_real'], res_df[yCol+'_pred'])
        print(f"{yCol} >> MAPE: {mape:.4f}, zero_mape: {zero_mape:.4f}, smape: {smape:.4f}")
    else:
        recall = recall_score(res_df[yCol+'_real'], res_df[yCol+'_pred'])
        accuracy = accuracy_score(res_df[yCol+'_real'], res_df[yCol+'_pred'])
        print(f"{yCol} >> reall: {recall:.4f}, accuracy: {accuracy:.4f}")

    # 학습된 모델의 성능 요약
    predictor.leaderboard()
    print(f'='*80,'\n')

### 2) 모델 선정

In [ ]:
# 선정된 모델명 리스트
# 기본물성 값 예측 최종 모델
trg_model_ls = ['LightGBM_BAG_L2']
# # 기본물성 불량여부 예측 최종 모델
trg_model_ls = ['XGBoost_BAG_L1']
# trg_model_ls = ['LightGBMXT_BAG_L1', 'LightGBM_BAG_L1', 'KNeighborsDist_BAG_L1', 'KNeighborsUnif_BAG_L1']
print(f'선정된 모델 : {trg_model_ls}')

### 3) 개별(LAB, MES-CMB, MES-FMB) 데이터의 최적 예측 모델(들) 선정 및 모델별 성능 요약

In [ ]:
# 물성값: regression, 불량여부: binary
problem_type_ = 'binary' # 'regression', 'binary'

# 모델 인덱스
file_suffix='combined'

if problem_type_ == 'regression':
    # 물성값 예측
    # yCols=['HS','SG','TS','EB']#'MNY','REHO_MIN','REHO_MAX','REHO_TS2','REHO_TC90','SCR']
    yCols=['MNY','REHO_MIN','REHO_MAX','REHO_TS2','REHO_TC90','SCR']
    data_type_ = 'SFT_regr'
    
    col_Nms = ['type','target_y']
    for trg_model in trg_model_ls:
        col_Nms = col_Nms + ['mape','mape0','smape']    
else:
    # 불량여부 예측
    yCols=['HS_RESULT','SG_RESULT','TS_RESULT','EB_RESULT']#,'MNY_RESULT','REHO_RESULT','SCR_RESULT']
    # yCols=['MNY_RESULT','REHO_RESULT','SCR_RESULT']
    data_type_ = 'SFT_clas'
    
    col_Nms = ['type','target_y']
    for trg_model in trg_model_ls:
        col_Nms = col_Nms + ['recall','accuracy','']
    
col_Nms.append('데이터수')
fin_df = pd.DataFrame(columns=col_Nms)

for yCol in yCols:    
    print(f'{yCol}!!!!')
    print(f'='*80)
    # 데이터셋 로드 및 모델 입출력 데이터 생성
    train = pd.read_csv(f'./final_dataset/{file_suffix}_data_{yCol}_fin.csv')
    X_train, X_test, y_train, y_test = train_test_split(train.drop(['ID', 'REAL_VAL'], axis=1), train[['REAL_VAL','TYPE']], test_size=0.2, random_state=seed)

    # 모델 로드    
    predictor = TabularPredictor.load(path=f'{data_type_}/{file_suffix}_{yCol}_models', verbosity = 0)
        
    types = X_test['TYPE'].unique().tolist()
    types.sort()
    display(types)
    
    for type_ in types:
        tmp_X_test = X_test[X_test['TYPE'] == type_]
        tmp_y_test = y_test[y_test['TYPE'] == type_]
        tmp_X_test = tmp_X_test.drop(columns=['TYPE'])
        tmp_y_test = tmp_y_test['REAL_VAL']

        result_ls = []
        for model_name in trg_model_ls:
            # 테스트 세트에 대한 예측
            y_pred = predictor.predict(tmp_X_test, model=model_name)

            res_df = pd.DataFrame()
            res_df[yCol+'_real'] = tmp_y_test
            res_df[yCol+'_pred'] = y_pred

            # 성능 평가
            if problem_type_ == 'regression':
                mape = mean_absolute_percentage_error(res_df[yCol+'_real'], res_df[yCol+'_pred'])
                zero_mape = mape_non_zero(res_df[yCol+'_real'], res_df[yCol+'_pred'])
                smape = smape_cal(res_df[yCol+'_real'], res_df[yCol+'_pred'])
                print(f"{type_}: {yCol} >> MAPE: {mape:.4f}, zero_mape: {zero_mape:.4f}, smape: {smape:.4f}")
                result_ls = result_ls + [mape, zero_mape, smape]
            else:
                recall = recall_score(res_df[yCol+'_real'], res_df[yCol+'_pred'])
                accuracy = accuracy_score(res_df[yCol+'_real'], res_df[yCol+'_pred'])
                print(f"{type_}: {yCol} >> reall: {recall:.4f}, accuracy: {accuracy:.4f}")
                result_ls = result_ls + [recall, accuracy, '']

        fin_df.loc[len(fin_df)] = [type_, yCol] + result_ls + [len(tmp_X_test)]
        # predictor.leaderboard()

fin_df.sort_values(['type'])

### 4) 통합 데이터의 최적 예측 모델(들) 선정 및 모델별 성능 요약

In [ ]:
# 물성값: regression, 불량여부: binary
problem_type_ = 'binary' # 'regression', 'binary'

# 모델 인덱스
file_suffix='combined'

if problem_type_ == 'regression':
    # 물성값 예측
    yCols=['HS','SG','TS','EB']#'MNY','REHO_MIN','REHO_MAX','REHO_TS2','REHO_TC90','SCR']
    # yCols=['MNY','REHO_MIN','REHO_MAX','REHO_TS2','REHO_TC90','SCR']
    data_type_ = 'SFT_regr'
    
    col_Nms = ['target_y']
    for trg_model in trg_model_ls:
        col_Nms = col_Nms + ['mape','mape0','smape']
else:
    # 불량여부 예측
    yCols=['HS_RESULT','SG_RESULT','TS_RESULT','EB_RESULT']#,'MNY_RESULT','REHO_RESULT','SCR_RESULT']
    # yCols=['MNY_RESULT','REHO_RESULT','SCR_RESULT']
    data_type_ = 'SFT_clas'
    
    col_Nms = ['target_y']
    for trg_model in trg_model_ls:
        col_Nms = col_Nms + ['recall','accuracy','']
    
col_Nms.append('데이터수')
fin_df = pd.DataFrame(columns=col_Nms)

for yCol in yCols:    
    print(f'{yCol}!!!!')
    print(f'='*80)
    # 데이터셋 로드 및 모델 입출력 데이터 생성
    train = pd.read_csv(f'./final_dataset/{file_suffix}_data_{yCol}_fin.csv')
    X_train, X_test, y_train, y_test = train_test_split(train.drop(['ID', 'REAL_VAL','TYPE'], axis=1), train[['REAL_VAL']], test_size=0.2, random_state=seed)

    # 모델 로드    
    predictor = TabularPredictor.load(path=f'{data_type_}/{file_suffix}_{yCol}_models', verbosity = 0)
        
    result_ls = []
    for model_name in trg_model_ls:
        # 테스트 세트에 대한 예측
        y_pred = predictor.predict(X_test, model=model_name)

        res_df = pd.DataFrame()
        res_df[yCol+'_real'] = y_test
        res_df[yCol+'_pred'] = y_pred

        # 성능 평가
        if problem_type_ == 'regression':
            mape = mean_absolute_percentage_error(res_df[yCol+'_real'], res_df[yCol+'_pred'])
            zero_mape = mape_non_zero(res_df[yCol+'_real'], res_df[yCol+'_pred'])
            smape = smape_cal(res_df[yCol+'_real'], res_df[yCol+'_pred'])
            print(f"{yCol} >> MAPE: {mape:.4f}, zero_mape: {zero_mape:.4f}, smape: {smape:.4f}")
            result_ls = result_ls + [mape, zero_mape, smape]
        else:
            recall = recall_score(res_df[yCol+'_real'], res_df[yCol+'_pred'])
            accuracy = accuracy_score(res_df[yCol+'_real'], res_df[yCol+'_pred'])
            print(f"{yCol} >> reall: {recall:.4f}, accuracy: {accuracy:.4f}")
            result_ls = result_ls + [recall, accuracy, '']

    fin_df.loc[len(fin_df)] = [yCol] + result_ls + [len(y_test)]
    # predictor.leaderboard()

fin_df

### 5) 최종적으로 선정된 모델 외 삭제 (모델 로딩 시간 단축)
<p style="font-weight:bold"> <span style="color:red">** 주의 : 실수로 모델 제거시, 다시 학습해야하므로 복사 해두고 진행하기 바랍니다.</span> </p>

In [ ]:
# # trg_model_ls = ['XGBoost_BAG_L1']
# trg_model_ls = ['LightGBM_BAG_L1']
print(f'선정된 모델 : {trg_model_ls}')

In [ ]:
# 물성값: regression, 불량여부: binary
problem_type_ = 'binary' # 'regression', 'binary'

# 모델 인덱스
file_suffix='combined'

if problem_type_ == 'regression':
    # 물성값 예측
    yCols=['HS','SG','TS','EB']#'MNY','REHO_MIN','REHO_MAX','REHO_TS2','REHO_TC90','SCR']
    # yCols=['MNY','REHO_MIN','REHO_MAX','REHO_TS2','REHO_TC90','SCR']
    data_type_ = 'SFT_regr'
else:
    # 불량여부 예측
    yCols=['HS_RESULT','SG_RESULT','TS_RESULT','EB_RESULT']#,'MNY_RESULT','REHO_RESULT','SCR_RESULT']
    # yCols=['MNY_RESULT','REHO_RESULT','SCR_RESULT']
    data_type_ = 'SFT_clas'

for yCol in yCols:
    print(yCol)
    predictor = TabularPredictor.load(path=f'{data_type_}/{file_suffix}_{yCol}_models', verbosity = 0)
    predictor.delete_models(models_to_keep=trg_model_ls, models_to_delete=None, dry_run=False)
    predictor.leaderboard()